# Intermediate Python for Engineers Day 5

Hi!

## Session 5: System automation

Session 5 gives you a tour of the amazing standard library and important 3rd-party tools for automating various systems-level tasks with Python:

- Concurrency and parallelism: opportunities and gotchas; dask
- Handling files & paths; dates, times, and IP addresses
- Compression; cryptographic hashing of file contents
- Automating local commands via subprocess
- Process and system monitoring with psutil
- Automatically creating config files via templating with Jinja2
- Parsing various config files: JSON, YAML, INI, XML (topics on request)


#### Please see the Zulip poll!

In [1]:
from threading import Thread
from time import sleep

In [2]:
def myfunc(n: int):
    sleep(n)

In [3]:
t = Thread(target=myfunc, args=(10,))
t.start()

In [4]:
t.join()

In [8]:
%%writefile mythreads2.py

from threading import Thread

count = 0
def counter():
    global count
    for i in range(10**6):
        count += 1
        count -= 1
    
NUM_THREADS = 1000
threads = [Thread(target=counter) for _ in range(NUM_THREADS)]
[t.start() for t in threads]
[t.join() for t in threads]
print(count)

Writing mythreads2.py


In [9]:
## Conclusion: you can create 1k threads but not 10k

In [10]:
import time
def countdown(n):
    while n > 0:
        print('Down', n)
        time.sleep(1)
        n -= 1

In [11]:
countdown(5)

Down 5
Down 4
Down 3
Down 2
Down 1


In [30]:
def countup(n):
    i = 0
    while i < n:
        print('Up', i)
        time.sleep(1)
        i += 1

In [31]:
countup(5)

Up 0
Up 1
Up 2
Up 3
Up 4


In [36]:
def countdown_gen(n):
    while n > 0:
        print('Down', n)
        time.sleep(4)
        n -= 1
        yield

def countup_gen(n):
    i = 0
    while i < n:
        print('Up', i)
        time.sleep(4)
        i += 1
        yield

In [37]:
countdown_gen(10)

<generator object countdown_gen at 0x7f1d0863d510>

In [38]:
list(zip(countdown_gen(10), countup_gen(10)))

Down 10
Up 0
Down 9
Up 1
Down 8
Up 2
Down 7
Up 3
Down 6
Up 4
Down 5
Up 5
Down 4
Up 6
Down 3
Up 7
Down 2
Up 8
Down 1
Up 9


[(None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None)]

In [39]:
def countdown_gen_sol(n):
    while n > 0:
        print('Down', n)
        yield n
        n -= 1

def countup_gen_sol(n):
    x = 0
    while x < n:
        print('Up', x)
        yield x
        x += 1

In [40]:
import time

g1 = countdown_gen_sol(10)
g2 = countup_gen_sol(10)

for i in range(10):
    next(g1)
    next(g2)
    time.sleep(0.99)

Down 10
Up 0
Down 9
Up 1
Down 8
Up 2
Down 7
Up 3
Down 6
Up 4
Down 5
Up 5
Down 4
Up 6
Down 3
Up 7
Down 2
Up 8
Down 1
Up 9


In [43]:
### ASYNC Solution! ###

In [48]:
import asyncio

In [49]:
async def a_countdown_gen(n):
    while n > 0:
        print('Down', n)
        n -= 1
        await asyncio.sleep(1)

async def a_countup_gen(n):
    i = 0
    while i < n:
        print('Up', i)
        await asyncio.sleep(1)
        i += 1

In [50]:
down = a_countdown_gen(5)

<ipython-input-50-fd969fa4d739>:1: RuntimeWarning: coroutine 'countdown_gen' was never awaited
  down = a_countdown_gen(5)


In [51]:
type(down)

coroutine

In [56]:
!pip install curio

Processing /home/data/.cache/pip/wheels/b0/c9/96/a066ccd667d7b82ececb5bbdcf955e64f2c609d5a6c2353f30/curio-1.4-py3-none-any.whl


In [57]:
%%writefile fib_in_process.py

import curio

def fib(n):
    if n <= 2:
       return 1
    else:
       return fib(n-1) + fib(n-2)

async def main():
    result = await curio.run_in_process(fib, 40)
    print(result)

curio.run(main())

Overwriting fib_in_process.py


In [58]:
!python fib_in_process.py


Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/curio/kernel.py", line 823, in run
    return kernel.run(corofunc, *args)
  File "/opt/conda/lib/python3.8/site-packages/curio/kernel.py", line 173, in run
    raise ret_exc
  File "/opt/conda/lib/python3.8/site-packages/curio/kernel.py", line 737, in kernel_run
    trap = current.send(current._trap_result)
  File "/opt/conda/lib/python3.8/site-packages/curio/task.py", line 167, in send
    return self._send(value)
  File "/opt/conda/lib/python3.8/site-packages/curio/task.py", line 171, in _task_runner
    return await coro
  File "/home/jovyan/fib_in_process.py", line 11, in main
    result = await curio.run_in_process(fib, 40)
  File "/opt/conda/lib/python3.8/site-packages/curio/workers.py", line 213, in run_in_process
    return await worker.apply(callable, args)
  File "/opt/conda/lib/python3.8/site-packages/curio/workers.py", line 381, in apply
    self._launch()
  File "/opt/conda/lib/python3.8/site

In [54]:
import requests
import json 
key = 'AZ0IT0GT66HJ1U36'
ticker = 'TSLA'
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&apikey={}'.format(ticker, key)
response = requests.get(url)
print(response.json()) 

{'Meta Data': {'1. Information': 'Daily Time Series with Splits and Dividend Events', '2. Symbol': 'TSLA', '3. Last Refreshed': '2020-10-22', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2020-10-22': {'1. open': '441.9200', '2. high': '445.2300', '3. low': '424.5100', '4. close': '425.7900', '5. adjusted close': '425.7900', '6. volume': '39993191', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2020-10-21': {'1. open': '422.7000', '2. high': '432.9500', '3. low': '421.2500', '4. close': '422.6400', '5. adjusted close': '422.6400', '6. volume': '32370461', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2020-10-20': {'1. open': '431.7500', '2. high': '431.7500', '3. low': '419.0501', '4. close': '421.9400', '5. adjusted close': '421.9400', '6. volume': '31656289', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2020-10-19': {'1. open': '446.2400', '2. high': '447.0000', '3. low': '428.8700', '4. 

In [60]:
response.url

'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=TSLA&apikey=AZ0IT0GT66HJ1U36'

In [66]:
import ipaddress

In [68]:
ip = '192.168.1.256'

In [69]:
try:
    addr = ipaddress.IPv4Address(ip)
except ipaddress.AddressValueError:
    print("Invalid address")
else:
    print("Valid address")

Invalid address


In [61]:
!pip install netifaces



Processing /home/data/.cache/pip/wheels/9b/51/d5/d4682860392414da28ba3c5ffac0243983399b868a9cced6df/netifaces-0.10.9-cp38-cp38-linux_x86_64.whl


In [62]:
import netifaces
interfaces = netifaces.interfaces()
netifaces.ifaddresses('eth0')

{17: [{'addr': '3e:dc:1e:0b:87:36', 'broadcast': 'ff:ff:ff:ff:ff:ff'}],
 2: [{'addr': '192.168.161.116',
   'netmask': '255.255.255.255',
   'broadcast': '192.168.161.116'}]}

In [63]:
for iface in interfaces:
    ipaddrs = netifaces.ifaddresses(iface)
    if netifaces.AF_INET in ipaddrs:
        ipaddr_desc = ipaddrs[netifaces.AF_INET][0]
        print(f"Network interface: {iface}")
        if 'addr' in ipaddr_desc:
            print(f"\tIP address: {ipaddr_desc['addr']}")
        if 'netmask' in ipaddr_desc:
            print(f"\tNetmask: {ipaddr_desc['netmask']}")

Network interface: lo
	IP address: 127.0.0.1
	Netmask: 255.0.0.0
Network interface: eth0
	IP address: 192.168.161.116
	Netmask: 255.255.255.255


In [70]:
loopback = ipaddress.IPv4Interface('127.0.0.1')

In [71]:
loopback


IPv4Interface('127.0.0.1/32')

In [72]:
netifaces.ifaddresses('eth0')


{17: [{'addr': '3e:dc:1e:0b:87:36', 'broadcast': 'ff:ff:ff:ff:ff:ff'}],
 2: [{'addr': '192.168.161.116',
   'netmask': '255.255.255.255',
   'broadcast': '192.168.161.116'}]}

In [73]:
loopback.is_loopback
loopback.is_private
loopback.is_multicast

False

In [77]:
example = ipaddress.ip_network("192.168.0.0/16")
example.netmask
example.broadcast_address
example.network_address
example.num_addresses


65536

In [79]:
### Consider the network "192.168.5.0/28".

In [80]:
net = ipaddress.ip_network('192.168.5.0/28')

In [82]:
from __future__ import annotations

In [83]:
def get_subnet_addresses(address: ipaddress.IPv4Address, mask_length: int):
    """
    Get all IP addresses on the subnet that is defined by `address`
    with the given subnet mask length
    """
    if not isinstance(address, ipaddress.IPv4Address):
        raise TypeError(f"address must be of type ipaddress.IPv4Address, got {type(address)}")
    iface_str = f"{str(address)}/{mask_length}"
    iface = ipaddress.IPv4Interface(iface_str)
    net = iface.network
    addresses = list(net.hosts())
    return addresses

In [86]:
address = ipaddress.IPv4Address("192.168.1.0")

In [88]:
subnet_addresses = get_subnet_addresses(address, 28)

In [89]:
subnet_addresses

[IPv4Address('192.168.1.1'),
 IPv4Address('192.168.1.2'),
 IPv4Address('192.168.1.3'),
 IPv4Address('192.168.1.4'),
 IPv4Address('192.168.1.5'),
 IPv4Address('192.168.1.6'),
 IPv4Address('192.168.1.7'),
 IPv4Address('192.168.1.8'),
 IPv4Address('192.168.1.9'),
 IPv4Address('192.168.1.10'),
 IPv4Address('192.168.1.11'),
 IPv4Address('192.168.1.12'),
 IPv4Address('192.168.1.13'),
 IPv4Address('192.168.1.14')]

In [ ]:
### Cyberpandas  Gives you an IPType as a dtype

In [223]:
import asyncio

async def count_words_async(file_path):
    with open(file_path, mode='rb') as file:
        proc = await asyncio.create_subprocess_exec(
            'wc',
            stdin=file,
            stdout=asyncio.subprocess.PIPE,
            stderr=asyncio.subprocess.PIPE)

        await proc.wait()
        # This works:
        #     result = await proc.stdout.read()
        #     return result.decode('utf8').strip()
        # But the docs seem to recommend this instead:
        stdout, stderr = await proc.communicate()
        return stdout.decode('utf8').strip()

async def main():
    result = await count_words_async('/Data/airfares.txt')
    print(result)

await main()

4177  37593 304921


In [100]:
import socket

In [101]:
socket.gethostbyname('pythoncharmers.com')

'13.237.211.31'

In [102]:
import dns.resolver

In [105]:
answers = dns.resolver.resolve('google.com', 'A')

In [106]:
for record_data in answers:
    print('google.com', record_data.address)

google.com 172.217.167.110


In [108]:
answers = dns.resolver.resolve('google.com', 'AAAA')

In [109]:
for record_data in answers:
    print('google.com', record_data.address)

google.com 2404:6800:4006:812::200e


In [110]:
socket.getaddrinfo('www.imc.com', 80, type=socket.SOCK_STREAM)


[(<AddressFamily.AF_INET: 2>,
  <SocketKind.SOCK_STREAM: 1>,
  6,
  '',
  ('54.206.19.82', 80)),
 (<AddressFamily.AF_INET6: 10>,
  <SocketKind.SOCK_STREAM: 1>,
  6,
  '',
  ('2406:da1c:6aa:c000:cfd6:c818:194:93fa', 80, 0, 0))]

In [111]:
goog_ip6 = ipaddress.IPv6Address(record_data.address)


In [112]:
goog_ip6

IPv6Address('2404:6800:4006:812::200e')

In [113]:
goog_ip6.is_global


True

In [114]:
name1 = dns.name.from_text('www.imc.com')


In [115]:
name1

<DNS name www.imc.com.>

In [116]:
name2 = dns.name.from_text('imc.com')


In [117]:
name1.is_subdomain(name2)


True

In [118]:
name1.canonicalize()


<DNS name www.imc.com.>

In [119]:
import subprocess


In [120]:
subprocess.call(['hostname'])


0

In [125]:
subprocess.call(['hostname', '-x'])

255

In [126]:
subprocess.check_output(['hostname'])


b'jupyter-faek-2esoussi-40imc-2ecom\n'

In [127]:
subprocess.check_output(['hostname']).decode('utf8')


'jupyter-faek-2esoussi-40imc-2ecom\n'

In [131]:
subprocess.call(['hostname', '-I'])


0

In [132]:
subprocess.check_output(['hostname'])

b'jupyter-faek-2esoussi-40imc-2ecom\n'

In [136]:
try:
    subprocess.check_output(['hostname', '-X'])
except subprocess.CalledProcessError as e:
    print(e.stderr)
    print(e.stdout)

None
b'Usage: hostname [-b] {hostname|-F file}         set host name (from file)\n       hostname [-a|-A|-d|-f|-i|-I|-s|-y]       display formatted name\n       hostname                                 display host name\n\n       {yp,nis,}domainname {nisdomain|-F file}  set NIS domain name (from file)\n       {yp,nis,}domainname                      display NIS domain name\n\n       dnsdomainname                            display dns domain name\n\n       hostname -V|--version|-h|--help          print info and exit\n\nProgram name:\n       {yp,nis,}domainname=hostname -y\n       dnsdomainname=hostname -d\n\nProgram options:\n    -a, --alias            alias names\n    -A, --all-fqdns        all long host names (FQDNs)\n    -b, --boot             set default hostname if none available\n    -d, --domain           DNS domain name\n    -f, --fqdn, --long     long host name (FQDN)\n    -F, --file             read host name or NIS domain name from given file\n    -i, --ip-address       addr

In [137]:
subprocess.check_output(['hostname', '-I'])

b'192.168.161.116 \n'

In [138]:
print(subprocess.check_output('ifconfig').decode('utf8'))

eth0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 9001
        inet 192.168.161.116  netmask 255.255.255.255  broadcast 0.0.0.0
        ether 3e:dc:1e:0b:87:36  txqueuelen 0  (Ethernet)
        RX packets 50734  bytes 22001196 (22.0 MB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 55008  bytes 110140706 (110.1 MB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

lo: flags=73<UP,LOOPBACK,RUNNING>  mtu 65536
        inet 127.0.0.1  netmask 255.0.0.0
        loop  txqueuelen 1000  (Local Loopback)
        RX packets 4196  bytes 1625273 (1.6 MB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 4196  bytes 1625273 (1.6 MB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0




In [139]:
proc = subprocess.Popen('ifconfig')

In [140]:
type(proc)

subprocess.Popen

In [144]:
proc.poll()  # check if the child process is still running

0

In [150]:
proc = subprocess.Popen(['sleep', '10'])

In [152]:
proc.poll() # check if process is alive 

-9

In [155]:
proc.wait()

-9

In [154]:
proc.kill()

In [156]:
proc.terminate()

In [157]:
word_counter = subprocess.Popen(['wc'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [158]:
word_counter.poll()

In [159]:
output, error = word_counter.communicate('Good morning Amsterdam!'.encode('utf8'))

In [161]:
output


b'      0       3      23\n'

In [162]:
error

b''

In [172]:
open('/Data/maugham_the_verger.txt')


<_io.TextIOWrapper name='/Data/maugham_the_verger.txt' mode='r' encoding='UTF-8'>

In [177]:
proc.stdin.write(f.read())

AttributeError: 'NoneType' object has no attribute 'write'

In [178]:
with open('/Data/maugham_the_verger.txt') as f:
    word_count = subprocess.Popen(['wc'], stdin=f, stdout=subprocess.PIPE)

In [179]:
word_count.poll()

0

In [180]:
word_count.stdout.read()

b'  309  2823 15206\n'

In [193]:
subprocess.PIPE

-1

In [197]:
pip_list = subprocess.Popen(['pip', 'list'], stdout=subprocess.PIPE)
output = subprocess.Popen(('grep', 'cookie'), stdin=pip_list.stdout, stdout=subprocess.PIPE)
pip_list.wait()

0

In [198]:
output.stdout.read()

b'cookiecutter                      1.7.2\n'

In [199]:
!mkdir templates

In [200]:
%%writefile templates/system_status.txt
This will be a system status report

{{ free_space }}
{{ used_space }}

Writing templates/system_status.txt


In [201]:
from jinja2 import Environment, FileSystemLoader
env = Environment(
    loader=FileSystemLoader('templates')
)

In [202]:
template = env.get_template('system_status.txt')

In [203]:
template.render()

'This will be a system status report\n\n\n'

In [204]:
print(template.render(free_space=123, used_space=456))

This will be a system status report

123
456


In [206]:
%%writefile templates/system_status.txt
This will be a system status report

{% for filename, filesize in csvs.items() %}
{{ filename }}, {{ filesize | abs }}
{% endfor %}

Overwriting templates/system_status.txt


In [207]:
csvs = {'name.csv': 123}

In [208]:
template = env.get_template('system_status.txt')
template.render(csvs=csvs)

'This will be a system status report\n\n\nname.csv, 123\n'

In [210]:
import humanize

In [211]:
humanize.naturalsize(123)

'123 Bytes'

In [212]:
env.filters['naturalsize'] = humanize.naturalsize

In [213]:
from glob import glob
import os.path

In [214]:
csvs = {path: os.path.getsize(path) for path in glob('/Data/*.csv')}

In [226]:
%%writefile templates/system_status.txt
This will be a system status report

{%- for filename, filesize in csvs.items() %}
{{ filename }}, {{ filesize | naturalsize }}{# the vertical bar is to pass the value on the left to a filter #}
{%- endfor %}

Overwriting templates/system_status.txt


In [227]:
template = env.get_template('system_status.txt')

In [228]:
print(template.render(csvs=csvs))

This will be a system status report
/Data/peak_sunlight_hours.csv, 4.4 kB
/Data/sampled_tesla.csv, 6.7 kB
/Data/surnames.csv, 8.6 MB
/Data/melb_monthly_weather_086071.csv, 47.3 kB
/Data/interest_inflation_aud.csv, 3.2 kB
/Data/bank-small.csv, 919.0 kB
/Data/gapminder-health-income.csv, 5.3 kB
/Data/abalone.csv, 192.0 kB
/Data/calories.csv, 48.2 kB
/Data/LIBOR.csv, 168.9 kB
/Data/heights_and_weights.csv, 36.1 kB
/Data/darwin-temp.csv, 31.0 kB
/Data/tips.csv, 7.9 kB
/Data/gapminderDataFiveYear.csv, 99.8 kB
/Data/countries.csv, 7.0 kB
/Data/melbourne-temp.csv, 13.7 kB
/Data/calories_clean.csv, 48.2 kB
/Data/world_bank_countries_data.csv, 87.6 kB
/Data/fed.csv, 15.7 kB
/Data/baby-names.csv, 7.4 MB
/Data/country_populations_by_year.csv, 171.8 kB
/Data/electricity_meter_usage_2017a.csv, 56.7 kB
/Data/public_holidays.csv, 35.2 kB
/Data/eight_schools.csv, 128 Bytes
/Data/wine.csv, 11.6 kB
/Data/oecd_stats.csv, 3.2 kB
/Data/Auto.csv, 20.5 kB
/Data/study_hours.csv, 154 Bytes
/Data/coal_mining_di